In [663]:
import pandas as pd
import ast
import numpy as np
import os
from matplotlib import pyplot as plt

In [664]:
df = pd.read_csv(r'C:\Users\zinhyeok\Developer\gbottomup\results\multi_point_GraphStep\20250908_104445\multiple_test_modelsingle3-1_d500_frequency_50_20250908_104445.csv')

In [665]:
df

,scenario,seed,model,True_tau,G_multi_tau,G_bottomup_candidates,G_bottomup_tau,G_multi_n_cps,G_bottomup_n_cps,G_multi_recall,G_bottomup_recall,G_multi_ari,G_bottomup_ari
0,modelsingle3-1,0,alpha_0.05c_2_topk_minobs4_numcp3,"[49, 99, 149]","[4, 46, 56, 73, 76, 133, 159]","[1, 5, 19, 25, 43, 49, 59, 69, 89, 123, 135, 1...","[69, 89, 169]",7,3,0.333333,0.333333,0.555325,0.489244
1,modelsingle3-1,0,alpha_0.05c_2_topk_minobs6_numcp3,"[49, 99, 149]","[4, 46, 56, 73, 76, 133, 159]","[41, 50, 56, 104, 131, 134, 149, 167]","[41, 50, 167]",7,3,0.333333,1.000000,0.555325,0.423993
2,modelsingle3-1,0,alpha_0.05c_2_topk_minobs8_numcp3,"[49, 99, 149]","[4, 46, 56, 73, 76, 133, 159]","[51, 75, 87, 91, 131, 139, 159, 187]","[87, 91, 159]",7,3,0.333333,0.333333,0.555325,0.553401
3,modelsingle3-1,0,alpha_0.05c_2_topk_minobs10_numcp3,"[49, 99, 149]","[4, 46, 56, 73, 76, 133, 159]","[4, 14, 24, 44, 49, 69, 104, 109, 144, 159, 169]","[44, 109, 169]",7,3,0.333333,1.000000,0.555325,0.624866
4,modelsingle3-1,1,alpha_0.05c_2_topk_minobs4_numcp3,"[49, 99, 149]","[49, 98, 148]","[9, 45, 49, 69, 71, 97, 133, 143, 155, 193]","[49, 97, 143]",3,3,1.000000,0.666667,0.973203,0.898180
...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,modelsingle3-1,98,alpha_0.05c_2_topk_minobs10_numcp3,"[49, 99, 149]","[14, 52, 99, 148, 177]","[14, 59, 64, 69, 114, 149, 179]","[64, 114, 149]",5,3,1.000000,0.333333,0.785217,0.691774
396,modelsingle3-1,99,alpha_0.05c_2_topk_minobs4_numcp3,"[49, 99, 149]","[49, 81, 99, 149]","[9, 17, 35, 145, 173]","[9, 173, 197]",4,3,1.000000,0.333333,0.918824,0.076267
397,modelsingle3-1,99,alpha_0.05c_2_topk_minobs6_numcp3,"[49, 99, 149]","[49, 81, 99, 149]","[5, 26, 32, 65, 107, 113, 149, 164]","[2, 5, 149]",4,3,1.000000,0.333333,0.918824,0.338629
398,modelsingle3-1,99,alpha_0.05c_2_topk_minobs8_numcp3,"[49, 99, 149]","[49, 81, 99, 149]","[7, 31, 63, 99, 147]","[3, 7, 31]",4,3,1.000000,0.666667,0.918824,0.097410


In [666]:
df['model'].unique()

array(['alpha_0.05c_2_topk_minobs4_numcp3',
       'alpha_0.05c_2_topk_minobs6_numcp3',
       'alpha_0.05c_2_topk_minobs8_numcp3',
       'alpha_0.05c_2_topk_minobs10_numcp3'], dtype=object)

In [667]:
def calculate_recall_with_tolerance(true_tau_list, predicted_tau_list, tolerance):
    true_positives = 0
    for true_tau in true_tau_list:
        is_found = False
        for predicted_tau in predicted_tau_list:
            if abs(true_tau - predicted_tau) <= tolerance:
                is_found = True
                break
        if is_found:
            true_positives += 1
    true_positives = min(true_positives, len(true_tau_list))
    return true_positives / len(true_tau_list) if len(true_tau_list) > 0 else 0

In [668]:
for col in ['True_tau', 'G_multi_tau', 'G_bottomup_candidates', 'G_bottomup_tau']:
    df[col] = df[col].apply(ast.literal_eval)

In [669]:
tolerances = [0, 2, 4, 5]
models = df['model'].unique()

In [670]:
results = {}

In [671]:
for model_name in models:
    model_df = df[df['model'] == model_name]
    recalls = []
    for tol in tolerances:
        recall_multi = model_df.apply(lambda row: calculate_recall_with_tolerance(row['True_tau'], row['G_bottomup_candidates'], tol), axis=1).mean()
        recalls.append(recall_multi)
    results[model_name] = recalls

In [672]:
recall_df = pd.DataFrame(results, index=[f'recall tol {t}' for t in tolerances])

In [673]:
# Baseline(SBS) 재현율 추가 (G_bottomup_tau 사용)
baseline_recalls = []
for tol in tolerances:
    baseline_recall = df.apply(lambda row: calculate_recall_with_tolerance(row['True_tau'], row['G_multi_tau'], tol), axis=1).mean()
    baseline_recalls.append(baseline_recall)
recall_df['Baseline(SBS)'] = baseline_recalls

In [674]:
model_name_mapping = {
    'alpha_0.05c_2_topk_minobs4_numcp3': 'Model A',
    'alpha_0.05c_2_topk_minobs6_numcp3': 'Model B',
    'alpha_0.05c_2_topk_minobs8_numcp3': 'Model C',
    'alpha_0.05c_2_topk_minobs10_numcp3': 'Model D',
}
recall_df = recall_df.rename(columns=model_name_mapping)

In [675]:
# 열 순서 변경 (Baseline을 맨 앞으로)
cols = ['Baseline(SBS)'] + [col for col in recall_df if col != 'Baseline(SBS)']
recall_df = recall_df[cols]

print(recall_df)

              Baseline(SBS)   Model A   Model B   Model C   Model D
recall tol 0       0.463333  0.193333  0.083333  0.100000  0.396667
recall tol 2       0.826667  0.390000  0.280000  0.346667  0.396667
recall tol 4       0.916667  0.530000  0.470000  0.436667  0.396667
recall tol 5       0.940000  0.530000  0.526667  0.436667  0.686667


In [676]:
recall_df.head()

,Baseline(SBS),Model A,Model B,Model C,Model D
recall tol 0,0.463333,0.193333,0.083333,0.100000,0.396667
recall tol 2,0.826667,0.390000,0.280000,0.346667,0.396667
recall tol 4,0.916667,0.530000,0.470000,0.436667,0.396667
recall tol 5,0.940000,0.530000,0.526667,0.436667,0.686667


In [677]:
recall_df = recall_df.round(3)

In [678]:
# 데이터프레임을 CSV 파일로 저장
recall_df.to_csv('recall_by_tolerance.csv')

In [679]:
def safe_literal_eval(item):
    if isinstance(item, str):
        try:
            return ast.literal_eval(item)
        except (ValueError, SyntaxError):
            return np.nan
    return item

In [680]:
df['G_bottomup_tau'] = df['G_bottomup_tau'].apply(safe_literal_eval)

In [681]:
models

array(['alpha_0.05c_2_topk_minobs4_numcp3',
       'alpha_0.05c_2_topk_minobs6_numcp3',
       'alpha_0.05c_2_topk_minobs8_numcp3',
       'alpha_0.05c_2_topk_minobs10_numcp3'], dtype=object)

In [682]:
# 'seed' 열을 기준으로 중복된 데이터를 제거하여 각 seed별로 하나의 행만 남깁니다.
df_unique_seed = df.drop_duplicates(subset='seed').copy()

# 'Baseline(SBS)' 모델 시각화 (G_multi_tau 컬럼 사용)
sbs_cps_values = df_unique_seed['G_multi_tau'].explode()
sbs_cps_values = sbs_cps_values.dropna().astype(int)

value_counts_sbs = sbs_cps_values.value_counts().sort_index()

if not value_counts_sbs.empty:
    plt.figure(figsize=(12, 7))
    plt.bar(value_counts_sbs.index, value_counts_sbs.values, width=1.0, align='center')

    plt.title('Frequency Plot for Model: Baseline(SBS)', fontsize=18, fontweight='bold')
    plt.xlabel('Point', fontsize=14)
    plt.ylabel('Frequency', fontsize=14)

    plt.xlim(0, 200)
    plt.ylim(bottom=0)

    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)

    # 그래프를 파일로 저장
    plt.savefig('frequency_plot_Baseline(SBS).png')
    plt.close()
    print("Baseline(SBS) 모델에 대한 빈도수 플롯을 생성하고 'frequency_plot_Baseline(SBS)' 파일로 저장했습니다.")
else:
    print("No data to plot for model: Baseline(SBS)")


Baseline(SBS) 모델에 대한 빈도수 플롯을 생성하고 'frequency_plot_Baseline(SBS)' 파일로 저장했습니다.


In [683]:
for model_name in models:
    model_df = df[df['model'] == model_name].copy()
    
    all_cps_values = model_df['G_bottomup_tau'].explode()
    all_cps_values = all_cps_values.dropna().astype(int)
    
    value_counts = all_cps_values.value_counts().sort_index()
    
    if value_counts.empty:
        print(f"No data to plot for model: {model_name}")
        continue

    plt.figure(figsize=(12, 7))
    
    plt.bar(value_counts.index, value_counts.values, width=1.0, align='center')
    
    display_name = model_name_mapping.get(model_name, model_name)
    plt.title(f'Frequency Plot for Model: {display_name}', fontsize=18, fontweight='bold')
    plt.xlabel('Point', fontsize=14)
    plt.ylabel('Frequency', fontsize=14)
    
    plt.xlim(0, 200)
    plt.ylim(bottom=0)
    
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)
    
    # 그래프를 파일로 저장
    plt.savefig(f'frequency_plot_{display_name.replace(" ", "_")}.png')
    plt.close() # 메모리 누수 방지를 위해 plot을 닫아줍니다.

print("각 모델에 대한 빈도수 플롯을 생성하고 파일로 저장했습니다.")

각 모델에 대한 빈도수 플롯을 생성하고 파일로 저장했습니다.
